In [ ]:
import re
import numpy as np
import pandas as pd
import gensim
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense,AveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1
from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GRU, Conv1D, MaxPooling1D, Flatten, Attention, Input

In [ ]:
data=pd.read_csv(r'D:\CODING\Python\NLP\END_SEM\IMDB Dataset.csv')

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

data['review'] = data['review'].apply(preprocess_text)

data['label'] = data['sentiment'].map({'positive': 1, 'negative': 0})


In [ ]:
def load_glove(glove_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        word_embeddings = {}
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_embeddings[word] = vector
    return word_embeddings

def generate_embedding_matrix(word_vector, embedding_model, embedding_type, max_length=200):
    if embedding_type == 'glove':
        embedding_dim = len(next(iter(embedding_model.values())))
        embedding_matrix = np.zeros((max_length, embedding_dim))
        for i, word in enumerate(word_vector):
            if word in embedding_model:
                embedding_matrix[i] = embedding_model[word]
            if i >= max_length - 1:
                break
    else:
        word_embeddings = []
        for word in word_vector:
            if word in embedding_model.wv:
                word_embedding = embedding_model.wv[word]
                word_embeddings.append(word_embedding)
        embedding_matrix = np.array(word_embeddings)
        if len(embedding_matrix) < max_length:
            embedding_matrix = np.pad(embedding_matrix, ((0, max_length - len(embedding_matrix)), (0, 0)), mode='constant')
        else:
            embedding_matrix = embedding_matrix[:max_length, :]
    return np.sum(embedding_matrix, axis=0)

def train_word2vec(corpus, vector_size=100, window=10, min_count=1, workers=4, sg=1):
    return Word2Vec(sentences=corpus, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg)

def train_fasttext(corpus, vector_size=100, window=10, min_count=1, workers=4, sg=1):
    return FastText(sentences=corpus, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg)

def choose_embedding_model(corpus, model_type, glove_file):
    if model_type == 'word2vec':
        model = train_word2vec(corpus)
    elif model_type == 'fasttext':
        model = train_fasttext(corpus)
    elif model_type == 'glove' and glove_file is not None:
        model = load_glove(glove_file)
    else:
        raise ValueError("Invalid model type or missing GloVe file.")
    return model


In [ ]:
tokenized_corpus=data['review'].to_list()

In [ ]:
embedding_type = 'glove' 
embedding_model = choose_embedding_model(tokenized_corpus, model_type=embedding_type, glove_file=r'D:\CODING\Python\NLP\END_SEM\glove\eng\glove.6B.100d.txt')
data['embedding_matrix'] = data['review'].apply(lambda x: generate_embedding_matrix(x, embedding_model, embedding_type))
X = np.array(data['embedding_matrix'].tolist())
y=data['label'] 

In [ ]:
y_one_hot = to_categorical(data['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)


In [ ]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# BiLSTM (Bidirectional LSTM)
def create_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128), input_shape=input_shape))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# GRU
def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN-LSTM
def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(LSTM(128))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN-BiLSTM
def create_cnn_bilstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN-GRU
def create_cnn_gru_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(GRU(128))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# LSTM with Attention
def create_lstm_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    lstm_out = LSTM(128, return_sequences=True)(inputs)
    attention = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention = tf.reduce_mean(attention, axis=1)
    outputs = Dense(2, activation='softmax')(attention)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN with Attention
def create_cnn_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    cnn_out = Conv1D(64, 3, activation='relu')(inputs)
    cnn_out = MaxPooling1D(2)(cnn_out)
    attention = tf.keras.layers.Attention()([cnn_out, cnn_out])
    attention = tf.reduce_mean(attention, axis=1)
    outputs = Dense(2, activation='softmax')(attention)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# BiLSTM with Attention
def create_bilstm_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    bilstm_out = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    attention = tf.keras.layers.Attention()([bilstm_out, bilstm_out])
    attention = tf.reduce_mean(attention, axis=1)
    outputs = Dense(2, activation='softmax')(attention)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# GRU with Attention
def create_gru_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    gru_out = GRU(128, return_sequences=True)(inputs)
    attention = tf.keras.layers.Attention()([gru_out, gru_out])
    attention = tf.reduce_mean(attention, axis=1)
    outputs = Dense(2, activation='softmax')(attention)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
models = {
    "LSTM": create_lstm_model,
    "BiLSTM": create_bilstm_model,
    "GRU": create_gru_model,
    "CNN-LSTM": create_cnn_lstm_model,
    "CNN-BiLSTM": create_cnn_bilstm_model,
    "CNN-GRU": create_cnn_gru_model,
    # "LSTM-Attention": create_lstm_attention_model,
    # "CNN-Attention": create_cnn_attention_model,
    # "BiLSTM-Attention": create_bilstm_attention_model,
    # "GRU-Attention": create_gru_attention_model
}

input_shape = (X_train.shape[1], 1)

results = {}

for name, create_model in models.items():
    model = create_model(input_shape)
    print(f"Training {name} model...")
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)
    score = model.evaluate(X_test, y_test, verbose=0)
    results[name] = score
    print(f"{name} Test Accuracy: {score[1]}")